# Hidden markov model
> 
- toc: true 
- badges: true
- comments: true
- categories: [jupyter]
- image: images/svi.jpg


HMM is a Markov model whose states are not directly observed; instead, each state is characterized by a probability distribution function modeling the observation corresponding to that state. HMM has been extensively used in temporal pattern recognition such as speech, handwriting, gesture recognition, robotics, biological sequences, and recently in energy disaggregation. This tutorial will introduce the basic concept of HMM.

There are two variables in HMM: observed variables and hidden variables where the sequences of hidden variables form a Markov process, as shown in the figure below. 

![](my_icons/hmm.png)

In the context of NILM, the hidden variables are used to model states(ON, OFF, standby etc.) of individual appliances, and the observed variables are used to model the electric usage. HMMs have been widely used in most of the recently proposed NILM approach because it represents well the individual appliance internal states which are not directly observed in the targeted energy consumption.

A typical HMM is characterised by the following: 

- The finite set of hidden states $S $ (e.g ON, stand-by, OFF, etc.) of an appliance, $S = \{s_1, s_2....,s_N\} $. 
- The finite set of $M $ observable symbol $Y $ per states (power consumption) observed in each state, $Y = \{y_1, y_2....,y_M\} $. The observable symbol $Y $ can be discrete or a continuous set. 
- The transition matrix $\mathbf{A}=\{a_{ij},1\leq i,j \geq N\} $ represents the probability of moving from state $s_{t-1}=i $ to $s_t =j $such that: $a_{ij} = P(s_{t} =j \mid s_{t-1}=i) $, with $a_{ij} \leq 0 $ and where $s_t $ denotes the state occupied by the system at time $t $. The matrix $\mathbf{A} $ is $ N x N $.
- The emission matrix $\mathbf{B} =\{b_j(k)\} $ representing the probability of emission of symbol $k\in  Y $ when system state is $s_t=j $ such that: $b_j(k) = p(y_t = k \mid s_t=j)$ . The matrix $\mathbf{B} $ is an $N x M $. The emission probability can be discrete or continous distribution. If the emission is descrete a multinomial distribution is used and multivariate Gaussian distribution is usually used for continous emission.
- And the initial state probability distribution $\mathbf{\pi} = \{\pi_i \ldots \pi_N\} $ indicating the probability of each state of the hidden variable at $t = 1 $ such that, $\pi _i = P(q_1 = s_i), 1 \leq i \geq N $.

The complete HMM specification requires;

1. Finite set of hidden states $N $ and observation symbols $M$
2. Length of observation seqences $T$ and
3. Specification of three probability measures $ \mathbf{A}, \mathbf{B}$ and $\mathbf{\pi} $
The set of all HMM model parameters is represented by $\mathbf{\lambda} =(\pi, A, B)$. Since $S$ is not observed, the likelihood functions $Y$ is given by the joint distribution of $Y$ and $S$ over all possible states. 
 $$
P(Y \mid \lambda) = \sum P(Y, S \mid \lambda)
 $$
where 
$$
P(Y,S \mid \lambda) = P(Y \mid S,\lambda)P(S \mid \lambda)
$$
Note that $y_t$ is independent and identically distributed given state sequence $S = \{s_1, \ldots s_N\}$. Also each state at time $t$ depend on the state at its previous time $t-1$. Then
 $$
P(Y \mid S, \lambda) = \prod_{t=1}^T P(y_t \mid s_t)
 $$
Similarly
 $$
P(S \mid \lambda) = \pi _{s_1} \prod _{t=2}^T a_{ij}
 $$
The joint probability is therefore:
 $$
P(Y \mid \lambda) = \pi _{s_1}P(y_1 \mid s_1) \sum \prod_{t=2}^T a_{ij} P(y_t \mid s_t)
 $$

## Three main problems in HMMs

When applying HMM to a real-world problem, three essential issues must be solved. 

1. Evaluation Problem: Given HMM parameters $\lambda$ and the observation sequence $Y = \{Y_1, Y_2....,Y_M\}$, find $P(Y \mid \lambda)$ the likelihood of the observation sequence $Y$ given the model $\lambda$. This problem gives a score on how well a given model matches a given observation and thus allows you to choose the model that best matches the observation.
2. Decoding Problem: Given HMM parameters $\lambda$ and the observation sequence $Y = \{Y_1, Y_2....,Y_M\}$, find an optimal state sequence $S = \{S_1, S_2....,S_N\}$ which best explain the observation.This problem attempts to cover the hidden part of the model.
3. Learning Problem: Given the observation sequence $Y = \{Y_1, Y_2....,Y_M\}$, find the model parameters $\lambda$ that maximize $P(Y \mid \lambda)$.This problem attempts to optimize the model parameters to describe the model.

The first and the second problem can be solved by the dynamic programming algorithms known as the Viterbi algorithm and the Forward-Backward algorithm, respectively. The last one can be solved by an iterative Expectation-Maximization (EM) algorithm, known as the Baum-Welch algorithm. We will discuss the first and the second problem in this post.

## Solution to Problem 1
A straight forward way to solve this problem is to find $P(Y \mid S, \lambda)$ for fixed state sequences $S = \{s_1,...s_T \}$ and then sum up over all possible states. This is generally infeasible since it requires about $2TN^T$ multiplications. However, the problem can be efficiently solved by using the forward algorithm as follows:
 
### The forward-backward Algorithm
Let us define the **forward variable** 
$$
\alpha _t(i)=P(y_1,\ldots y_t, s_t=i \mid \lambda)
$$
the probability of the partial observation sequences $y_1 \ldots y_t$ up to time $t$ and the state $s_t =i$ at time $t$ given the model ${\lambda}$. We also define an emission probability given HMM state $i$ at time $t$ as $b_i(y_t)$.

#### Forward-Algorithm
- **Initilization** 

    Let
    $$
    \begin{aligned}
    \alpha _1(i)&=P(y_1, s_1=i \mid \lambda) \\
     & = P(y_1 \mid s_1=i,\lambda)P(s_1=i \mid \lambda)\\
     &= \pi _i b_i(y_1) \text{ for } 1\leq i \geq N
    \end{aligned}
    $$

- **Induction**

    For $t=2,3...T$ and $1\leq i \geq N$, compute:
   
    \begin{align}
    \alpha _{t}(i) & = P(y_1 \ldots y_t, s_t=i \mid \lambda)\\
     &= \displaystyle \sum_{j=1}^{N} P(y_1 \ldots y_{t}, s_{t-1}=j,s_t=i \mid \lambda) \\
     &= \displaystyle \sum_{j=1}^{N} P(y_t \mid s_t=i, y_1,\ldots y_{t-1}, s_{t-1}=j, \lambda) \\
     & \times P(s_t=i \mid y_1 \ldots y_{t-1} \ldots , s_{t-1}=j, \lambda) \\
     & \times P(y_1 \ldots y_{t-1}, s_{t-1}=j,\lambda) \\
     & = P(y_t \mid s_t=i,\lambda)\displaystyle \sum_{j=1}^{N} P(s_t=i \mid s_{t-1}=j)\cdot P(y_1, \ldots y_{t-1}, s_{t-1}) \\
    & = b_i(y_{t})\displaystyle \sum_{j=1}^{N} \alpha _{t-1}(i)a_{ij} 
    \end{align} 

- **Termination**
    
    From $\alpha _t(i)=P(y_1,...y_t, s_t=i \mid \lambda)$, it cear that:
    $$ 
    P(Y \mid \lambda) = \displaystyle \sum_{i=1}^{N} P(y_1,\ldots y_T, s_T = i \mid \lambda) = \displaystyle \sum_{i=1}^{N}\alpha _T(i) 
    $$


### Backward Algorithm
This is the same as the forward algorithm discussed in the previous sectionexcept that it start at the end and works backward toward the beginning. We first define the **backward variable** $\beta_t(i)=P(y_{t+1},y_{t+2} \ldots y_{T} \mid s_t=i, {\lambda})$: probability of the partial observed sequence from $$t+1 $$ to the end at $$T$$ given state $$i$$ at time $t$ and the model $\lambda$. 

Then $\beta_t(i)$ can be recursively computed as follows.
- **Initialization**
  Let $\beta_{T}(i)= 1$, for $1 \leq i\geq N$
- **Induction**
  For $t =T-1, T-2,\ldots1$ for $1 \leq i\geq N$ and by using the sum and product rules, we can rewrite $\beta_t(j)$ as:
\begin{aligned}
\beta_t(i)&=P(y_{t+1},\ldots y_{T} \mid s_t=j, {\lambda}) \\
 &= \displaystyle \sum_{i=1}^{N} P(y_{t+1} \ldots y_T, s_{t+1}=i \mid s_t=j, \lambda) \\
 & = \displaystyle \sum_{i=1}^{N} P(y_{t+1} \ldots y_T, s_{t+1}=i, s_t=j, \lambda)\cdot P(s_{t+1}=i \mid s_t=j) \\
 &= \displaystyle \sum_{i=1}^{N} P(y_{t+2} \ldots y_T, s_{t+1}=i, \lambda)\cdot P(y_{t+1} \mid s_{t + 1}=i, \lambda)\cdot P(s_{t+1}=i \mid s_t=j) \\
 & = \displaystyle \sum_{i=1}^{N} a_{ij}b_i(y_{t+1})\beta _{t+1}(i)
\end{aligned}
- **Termination**
\begin{aligned}
\beta_{0} & = P(Y \mid \lambda) \\
& = \displaystyle \sum_{i=1}^{N} P(y_1,\ldots y_T, s_1=i) \\
&= \displaystyle \sum_{i=1}^{N} P(y_1,\ldots y_T \mid s_1=i)\cdot P(s_1=i) \\
& = \displaystyle \sum_{i=1}^{N} P(y_1 \mid s_1=i)\cdot P(y_2,\ldots y_T \mid s_1=i)\cdot P(s_1=i) \\
& = \displaystyle \sum_{i=1}^{N} \pi _i b_i(y_1)\beta _1(i)
\end{aligned}




In [146]:
import pyro
import pyro.distributions as dist
num_categories = 2
num_words = 3
num_supervised_data = 50
num_data = 80

pi = np.array([0.6, 0.4])  #initial probability 
transition_prior = torch.tensor([0.6, 0.4])
emission_prior = torch.tensor([0.4, 0.2, 0.4])
transition_prob = dist.Dirichlet(transition_prior).sample(torch.Size([num_categories]))
emission_prob = dist.Dirichlet(emission_prior).sample(torch.Size([num_categories]))

In [147]:
def equilibrium(mc_matrix):
    n = mc_matrix.size(0)
    return (torch.eye(n) - mc_matrix.t() + 1).inverse().matmul(torch.ones(n))

start_prob = equilibrium(transition_prob)

# simulate data
categories, words = [], []
for t in range(num_data):
    if t == 0 or t == num_supervised_data:
        category = dist.Categorical(start_prob).sample()
    else:
        category = dist.Categorical(transition_prob[category]).sample()
    word = dist.Categorical(emission_prob[category]).sample()
    categories.append(category)
    words.append(word)
categories, words = torch.stack(categories), torch.stack(words)

In [149]:
states = ('Rainy', 'Sunny')
observations = ('walk', 'shop', 'clean')

In [150]:
obs_seq=list(map(lambda y: observations[y], words.numpy()))
state_seq=list(map(lambda y: states[y], categories.numpy()))

In [160]:
transition_prob = pyro.sample("transition_prob", dist.Dirichlet(transition_prior)).log().unsqueeze(0)
transition_prob.shape

torch.Size([1, 2])

In [6]:
bob_says = np.array([0, 2, 1, 1, 2, 0])
print("Bob says:", ", ",list(map(lambda y: observations[y], bob_says)))

Bob says: ,  ['walk', 'clean', 'shop', 'shop', 'clean', 'walk']


In [ ]:
## Reference

1. [[Cheng Zhang,(2017)]](https://arxiv.org/abs/1711.05597):